# Joins columns from benchmarks and jobsarchive tables and outputs them for ML analysis

In [20]:
import pickle
import re
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

benchmark_indices = ['benchmarks-2017.04']
daysOfData=18

In [21]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

def time_filter(indices, days=1, until=0):
    if days == 0:
        return ["jobs_archive_*"]
    today = datetime.date.today()
    filtered = []
    datefmt = '%Y-%m-%d'
    for i in indices:
        day = re.sub(r'jobs_archive_', '', i).rstrip()
        day = datetime.datetime.strptime(day, datefmt).date()
        diff = today - day
        if until <= diff.days < days + until:
            filtered.append(i.rstrip())
    return filtered

selIndices = time_filter(indices,daysOfData)
job_indices = ','.join(selIndices)
print(job_indices)

jobs_archive_2017-04-07,jobs_archive_2017-04-08,jobs_archive_2017-04-09,jobs_archive_2017-04-10,jobs_archive_2017-04-11,jobs_archive_2017-04-12,jobs_archive_2017-04-13,jobs_archive_2017-04-14,jobs_archive_2017-04-15,jobs_archive_2017-04-16,jobs_archive_2017-04-17,jobs_archive_2017-04-18,jobs_archive_2017-04-19,jobs_archive_2017-04-20,jobs_archive_2017-04-21,jobs_archive_2017-04-22,jobs_archive_2017-04-23,jobs_archive_2017-04-24


In [22]:
benchmarks_query={
    "size": 0,
    "query":{
       "match_all": {}
    }
}

res = scan(es, query=benchmarks_query, index=benchmark_indices, scroll='5m', timeout="5m", size=1000)

data=[]
benchmarksPerSite={}
count = 0
for r in res:
    pr=r['_source']['profiles']
    mt=r['_source']['metadata']
    site=mt['ATLASSite']
    if site not in benchmarksPerSite:
        benchmarksPerSite[site]=0
    benchmarksPerSite[site]+=1
    if 'fastBmk' not in pr: 
        fastBmk = 0
    else:
        fastBmk = pr['fastBmk']['value']
    if 'whetstone' not in pr:
        ws = 0
    else:
        ws = pr['whetstone']['score']
    doc=[mt['PanDAID'],mt['bogomips'],mt['ip'],site,mt['mp_num'],mt['cpuname'],mt['meminfo'],fastBmk,ws]
    data.append(doc)
    if not count%10000:
        print(count)
#     if count>100: break
    count+=1

print(len(data))
    
benchmark=pd.DataFrame(data,columns=['pandaid','bogomips','ip','site','mpnum','cpuname','meminfo','fastBmk','whetstone'])
del data

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
122612


In [23]:
#print(benchmark.dtypes)
benchmark.head()
pids=set(benchmark['pandaid'].tolist())
print('total benchmarks:', len(pids))


total benchmarks: 122602


In [24]:
benchmark.to_csv('benchmark.csv',index=False)


In [25]:
# select only production jobs as only these can potentially have benchmark done. 
# prodsourcelabel = prod_test, rc_test, managed
job_query = {
    "size": 0,
    "_source": ["cpuconsumptiontime", "wall_time", "hs06","nevents","prodsourcelabel","processingtype","computingsite"],
    'query':{
            'bool':{
                   'should':[
                      # { "term": {"prodsourcelabel": "rc_test" } },
                      # { "term": {"prodsourcelabel": "prod_test" } },
                       { "term": {"prodsourcelabel": "managed" } }
                   ]
            }
    }
                
}

data=[]
scroll = scan(client=es, index=job_indices, query=job_query, scroll='5m', timeout="5m", size=10000)
count = 0
counttype={'managed':0,'rc_test':0,'prod_test':0}
jobsPerSite={}
for res in scroll:
    count += 1
    #print(res)
    #if count>3: break
    
    r=res['_source']
    
    site=r['computingsite']
    if site not in jobsPerSite:
        jobsPerSite[site]=0
    jobsPerSite[site]+=1
    
    if not count%100000: 
        print(count, ' selected:', counttype)
        #print(data)
    
    pid=int(res['_id'])
    if pid not in pids: continue
    
    counttype[r['prodsourcelabel']]+=1
    cpu=r['cpuconsumptiontime']
    wall=r['wall_time']
    nevents=r['nevents']
    if wall>0: cpueff = cpu/wall
    if nevents>0:
        wallPerEvent = wall/nevents
        cpuPerEvent = cpu/nevents
    else:
        wallPerEvent = 0
        cpuPerEvent =0
    doc=[pid,cpu,wall,r['hs06'],r['processingtype'],nevents, cpueff, wallPerEvent, cpuPerEvent]
    data.append(doc)

job=pd.DataFrame(data,columns=['pandaid','cputime','walltime','hs06','processingtype','nevents','cpueff', 'wallPerEvent', 'cpuPerEvent'])
del data

100000  selected: {'rc_test': 0, 'managed': 514, 'prod_test': 0}
200000  selected: {'rc_test': 0, 'managed': 1029, 'prod_test': 0}
300000  selected: {'rc_test': 0, 'managed': 1536, 'prod_test': 0}
400000  selected: {'rc_test': 0, 'managed': 2043, 'prod_test': 0}
500000  selected: {'rc_test': 0, 'managed': 2494, 'prod_test': 0}
600000  selected: {'rc_test': 0, 'managed': 2898, 'prod_test': 0}
700000  selected: {'rc_test': 0, 'managed': 3390, 'prod_test': 0}
800000  selected: {'rc_test': 0, 'managed': 3861, 'prod_test': 0}
900000  selected: {'rc_test': 0, 'managed': 4314, 'prod_test': 0}
1000000  selected: {'rc_test': 0, 'managed': 4756, 'prod_test': 0}
1100000  selected: {'rc_test': 0, 'managed': 5189, 'prod_test': 0}
1200000  selected: {'rc_test': 0, 'managed': 5694, 'prod_test': 0}
1300000  selected: {'rc_test': 0, 'managed': 6146, 'prod_test': 0}
1400000  selected: {'rc_test': 0, 'managed': 6581, 'prod_test': 0}
1500000  selected: {'rc_test': 0, 'managed': 7061, 'prod_test': 0}
16000

In [7]:
job.head()

,pandaid,cputime,walltime,hs06,processingtype,nevents,cpueff,wallPerEvent,cpuPerEvent
0,3326375042,29513,11863,96,reprocessing,2338,2.487819,5.073995,12.623182
1,3327803396,22947,3502,77,merge,64062,6.552541,0.054666,0.358200
2,3330689156,7696,2433,90,merge,22779,3.163173,0.106809,0.337855
3,3335362910,3260,4759,10,reprocessing,12652,0.685018,0.376146,0.257667
4,3334380752,650,2313,10,reprocessing,2152,0.281020,1.074814,0.302045


#### store all the jobs together

In [8]:
job.to_csv('job.csv',index=False)

#### store jobs split per processingtype

In [9]:
UniqueProcessingTypes = job.processingtype.unique()
print(UniqueProcessingTypes)

#create a dict for all the dataframes to be filled later
ProcessingType = {elem : pd.DataFrame for elem in UniqueProcessingTypes}

#filling up data frames
for key in ProcessingType.keys():
    job[:][job.processingtype == key].to_csv(key+'.csv',index=False)

['reprocessing' 'merge' 'simul' 'evgen' 'pmerge' 'recon' 'eventIndex'
 'pile' 'overlay']


In [10]:
#full = pd.merge(benchmark, job, on='pandaid')
#full.head()
#full.to_csv('benchmark_job.csv',index=False)

In [11]:
#for ps in benchmarksPerSite:
#    print(ps, benchmarksPerSite[ps])

for ps in jobsPerSite:
    if ps not in benchmarksPerSite and jobsPerSite[ps]>1000: 
        print(ps, jobsPerSite[ps])#,'       <<<<<<<<<<<<<')
    #else:
        #print(ps, jobsPerSite[ps],benchmarksPerSite[ps])

CONNECT_JETSTREAM 11731
BNL_PROD 209667
CONNECT_ICCHEPT3 11246
CONNECT_UIUC 1823
LRZ-LMU_MUC1_MCORE 10573
ALCF_Theta 1078
CONNECT_FRESNOSTATE 2120
LRZ-LMU_MUC_MCORE1 2137
CONNECT_KARST 5727
OU_OSCER_ATLAS_MCORE 1349
CONNECT_JETSTREAM_MCORE 6564
praguelcg2_fzu_MCORE 3320
CA-VICTORIA-WESTGRID-T2_MCORE 9601
CPPM_MCORE 9589
ORNL_Titan_MCORE 28284
UKI-NORTHGRID-MAN-HEP_MCOREVAC 4254
SiGNET 37535
RRC-KI-HPC2 5769
LRZ-LMU_C2PAP_MCORE 28587
OU_OSCER_ATLAS_OPP 12977
HPC2N 9141
SiGNET-NSC_MCORE 4554
CA-VICTORIA-WESTGRID-T2 25719
BNL_LOCAL 7876
BNL_PROD_MCORE 142802
HPC2N_MCORE 13338
CONNECT_ICCHEPT3_MCORE 2020
CONNECT_KARST_MCORE 1509
SiGNET_MCORE 30719
OU_OSCER_ATLAS 7786
NERSC_Edison_2 14932
Anselm_MCORE 99885


In [12]:
print (jobsPerSite)
#print (benchmarksPerSite)

{'LPSC_MCORE': 10441, 'RAL-LCG2_HIMEM_SL6': 235, 'SWT2_CPB': 42233, 'CERN-PROD_T0_8MCORE': 27219, 'pic_MCORE': 17526, 'UIO_MCORE_LOPRI': 3292, 'IN2P3-CC_MCORE_HIMEM': 11429, 'IFAE_MCORE': 8431, 'GRIF-LAL_HTCondor': 22806, 'BEIJING-TIANJIN-TH-1A_MCORE': 685, 'UNIBE-LHEP_CLOUD_MCORE': 485, 'RRC-KI_PROD': 1222, 'ATLAS_OPP_OSG-CLEMSON_PALMETTO': 6023, 'TOKYO': 50821, 'UKI-SCOTGRID-GLASGOW_SL6': 78996, 'UKI-NORTHGRID-MAN-HEP_SL6': 27722, 'UKI-SOUTHGRID-OX-HEP_MCORE': 9590, 'GRIF-LAL_HTCondor_MCORE': 12427, 'CA-MCGILL-CLUMEQ-T2': 10335, 'INFN-FRASCATI_MCORE': 10134, 'INFN-T1-BA': 32391, 'IAAS_MCORE': 12241, 'SLACXRD_LMEM': 9028, 'FZK-LCG2': 32117, 'NCG-INGRID-PT_SL6': 2290, 'CONNECT_JETSTREAM': 11731, 'BNL_PROD': 209667, 'IL-TAU-HEP': 2511, 'INFN-NAPOLI-SCOPE': 9121, 'praguelcg2_fzu_SCORE': 6652, 'LUNARC': 13128, 'UKI-SOUTHGRID-SUSX_SL6': 715, 'SFU-LCG2_MCORE': 4212, 'BU_ATLAS_Tier2_SL6': 43700, 'UKI-NORTHGRID-LIV-HEP_VAC': 2064, 'RAL-LCG2-ECHO_MCORE': 53985, 'CONNECT_ICCHEPT3': 11246, 'CONN